In [1]:
import altair as alt
import pandas as pd
from IPython.display import Markdown, display, HTML

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

GCS_PATH = "gs://calitp-analytics-data/data-analyses/sb125/fund_split/"

# SB125 Fund Split Analysis

## Question:
- How did RTAs split SB125 funds between operations and capital?

## Methodology:
- upload all avilable `SB125 fund request template` files to gcs
- examine all files for consistencies:
    - come with cleaning plan for inconsistent examples (files withot capital/operating columns)
- concat all rows across all files


## Notes:
- Some RTPAs did not submit a `SB125 fund request template.xlsx` file, but instead included an equivilent file their allocation package
- There are ~30 files to analyze, most of which need to be uniquely cleaned. All cleaning steps are done in the `fund_split.py` script and saved to GCS

# Read in Melt DF parquet from GCS

In [2]:
all_melt = pd.read_parquet(f"{GCS_PATH}all_fund_requests_melt.parquet")

In [10]:
display(all_melt.shape, list(all_melt.columns))

(2296, 8)

['rtpa',
 'implementing agenc-y/-ies',
 'project',
 'fund source',
 'capital/operation fy',
 'fund amount',
 'project type',
 'fiscal year']

In [4]:
by_source = all_melt.groupby(["fund source"]).agg({"fund amount": "sum", "rtpa": "nunique"}).reset_index()
by_type = all_melt.groupby(["project type"]).agg({"fund amount": "sum",}).reset_index()
by_year = all_melt.groupby(["fiscal year", "project type"]).agg({"fund amount": "sum"}).reset_index()
by_rtpa = all_melt.groupby(["rtpa"]).agg({"fund amount": "sum", "project": "nunique"}).reset_index()


In [14]:
by_source

,fund source,fund amount,rtpa
0,CMAQ,2300000,1
1,Farebox,400000,1
2,Measure V,500000,1
3,SGR,620000,1
4,TIRCP,2114010519,30
5,ZETCP,58874499,3
6,ZETCP (GGRF),275714380,26
7,ZETCP (PTA),27584170,23
8,`5307,400000,1
9,`5339,600000,1


# Charts

In [18]:
stack_chart = alt.Chart(by_year).mark_bar(point=True).encode(
    x="fiscal year", 
    y="fund amount", 
    color="project type",
    tooltip=["project type", "fund amount"],
).properties(width=600, height=400)


overall_chart = alt.Chart(by_type).mark_bar(point=True).encode(
    x="project type", 
    y="fund amount", 
    color="project type",
    tooltip=["project type", "fund amount"],
).properties(width=600, height=400)

source_chart = alt.Chart(by_source).mark_bar(point=True).encode(
    x="fund source", 
    y="fund amount", 
    #color="project type",
    tooltip=["fund source", "fund amount"],
).properties(width=600, height=400)


display(
    Markdown("## Overall split of SB125 allocation funds by capital/operating expenses"),
    overall_chart,
    by_type,
    Markdown("## SB125 allocation funds by capital/operating expenses, by fiscal year"),
    stack_chart,
    by_year.sort_values(by="project type"),
    Markdown("## Allocation funds by funding source"),
    source_chart
)

## Overall split of SB125 allocation funds by capital/operating expenses

alt.Chart(...)

,project type,fund amount
0,capital,1855348721
1,operating,625654847


## SB125 allocation funds by capital/operating expenses, by fiscal year

alt.Chart(...)

,fiscal year,project type,fund amount
0,23-24,capital,945242943
2,24-25,capital,812480147
4,25-26,capital,49092309
6,26-27,capital,48533322
1,23-24,operating,230807266
3,24-25,operating,169494114
5,25-26,operating,96177774
7,26-27,operating,129175693


## Allocation funds by funding source

alt.Chart(...)

# Draft Aggregations

In [7]:
def make_bar(data, x_axis, y_axis):
    chart = (
        alt.Chart(data)
        .mark_bar()
        .encode(
            x=x_axis,
            y=y_axis,
            #color= color_val,
        )
        .properties(width="container", height=400)
    )
    text = chart.mark_text(align="left", baseline="middle", dx=3).encode(
            text="fund amount"
    )
    return chart + text

In [8]:
make_bar(by_source[by_source["fund source"].isin(
    ["TIRCP", "ZETCP", "ZETCP (GGRF)", "ZETCP (PTA)"]
)], y_axis="fund amount", x_axis="fund source")

alt.LayerChart(...)